In [1]:
import main.Constants as Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


## Generating Chat Evaluation Quiz

In [2]:
from main.chat_eval import *
from main.utils import parse_response_table

In [3]:
response_gen = ResponseQuizGenerator(gsheet_name=Constants.SHEET_NAME, wks_name=Constants.RESPONSE_LOG_SHEET_NAME)

In [4]:
response_gen.generate_response_quiz(
    situation ='Meeting Someone New',
    num_phrases =2,
    complexity = "Low",
    tone = "Polite",
    temp = 0.7,
    model = "gpt-4o-mini"
)

,Prompt,PromptPinyin,Response,Complexity,Tone
1,你好，我叫...,"Nǐ hǎo, wǒ jiào...",,Low,Polite
2,很高兴认识你！,Hěn gāoxìng rènshì nǐ!,,Low,Polite


In [5]:
response_gen.provide_response(['你好！我叫 Tom!', '我也很高兴认识你'])

In [6]:
response_gen.evaluate_response(
    temp = 0.7,
    model = "gpt-4o-mini"
)

,Prompt,PromptPinyin,PromptMeaning,Response,ResponsePinyin,ResponseMeaning,Correctness,Naturalness,Contextual Appropriateness,Comment on Correction,Complexity,Tone
1,你好，我叫...,"Nǐ hǎo, wǒ jiào...","Hello, my name is...",你好！我叫 Tom!,Nǐ hǎo! Wǒ jiào Tom!,Hello! My name is Tom!,10,10,10,The response is perfect. No corrections needed.,Low,Polite
2,很高兴认识你！,Hěn gāoxìng rènshì nǐ!,Nice to meet you!,我也很高兴认识你,Wǒ yě hěn gāoxìng rènshì nǐ,I’m also very happy to meet you,10,10,10,The response is excellent. No corrections needed.,Low,Polite


In [7]:
response_gen.output_quiz_log()

#### Manual Run

In [3]:
client = OpenAI(
    api_key = Constants.API_KEY_OPENAI,
)

def get_completion(prompt, model="gpt-4o-mini", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response

In [4]:
prompt = get_prompt_to_gen_phrases_for_quiz(
        situation="Coming back from a holiday break",
        num_phrases=2, 
        complexity="Medium",
        tone="Casual",
    )

In [5]:
temp = 0.7
model = "gpt-4o-mini"

prompt = get_prompt_to_gen_phrases_for_quiz(
        situation="Coming back from a holiday break",
        num_phrases=2, 
        complexity="Medium",
        tone="Casual",
    )

sample_response_translation = (
    get_completion(
        prompt=prompt, model=model , temperature=temp))
content = sample_response_translation.choices[0].message.content


In [6]:
phrase_df = parse_response_table(content)
phrase_df['Response'] = ['很好!跟我太太放松了一下', '对不起，我不会说中文']

In [7]:
phrase_df

,Prompt,PromptPinyin,Response,Complexity,Tone
1,假期过得怎么样？,Jiàqī guò dé zěnme yàng?,很好!跟我太太放松了一下,Medium,Casual
2,又要开始工作了，真不想！,"Yòu yào kāishǐ gōngzuò le, zhēn bù xiǎng!",对不起，我不会说中文,Medium,Casual


In [16]:
temp = 0.7
sample_response_translation = (
    get_completion(
        prompt=get_prompt_convo_eval(phrase_df), model="gpt-4o-mini" , temperature=temp))
content = sample_response_translation.choices[0].message.content


In [17]:
print(content)

| Prompt                               | PromptPinyin                          | PromptMeaning                 | Response                          | ResponsePinyin                      | ResponseMeaning               | Correctness | Naturalness | Contextual Appropriateness | Comment on Correction                          | Complexity | Tone   |
|--------------------------------------|--------------------------------------|-------------------------------|-----------------------------------|-------------------------------------|-------------------------------|-------------|-------------|---------------------------|------------------------------------------------|------------|--------|
| 假期过得怎么样？                   | Jiàqī guò dé zěnme yàng?            | How was your holiday?         | 很好!跟我太太放松了一下          | Hěn hǎo! gēn wǒ tàitài fàngsōng le yīxià | Great! Relaxed with my wife. | 9           | 9           | 9                         | Very good response, just a bit more detail could enha

In [18]:
eval_df = parse_response_table(content)
eval_df

,Prompt,PromptPinyin,PromptMeaning,Response,ResponsePinyin,ResponseMeaning,Correctness,Naturalness,Contextual Appropriateness,Comment on Correction,Complexity,Tone
1,假期过得怎么样？,Jiàqī guò dé zěnme yàng?,How was your holiday?,很好!跟我太太放松了一下,Hěn hǎo! gēn wǒ tàitài fàngsōng le yīxià,Great! Relaxed with my wife.,9,9,9,"Very good response, just a bit more detail cou...",Medium,Casual
2,又要开始工作了，真不想！,"Yòu yào kāishǐ gōngzuò le, zhēn bù xiǎng!",I really don’t want to start working again!,对不起，我不会说中文,"Duìbùqǐ, wǒ bù huì shuō zhōngwén","Sorry, I can't speak Chinese.",3,3,2,The response is not relevant to the prompt. Yo...,Medium,Casual


## Generating Word Quiz

In [16]:
from main.quiz import *

temp = 0.7
dict_sheet_name = "Tua_List"
gsheet_name = "New Chinese Words"

word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=dict_sheet_name)
word_dict.head(2)

,Word Id,Word,Pinyin,Pinyin Simplified,Type,Word Category,Word Rarity,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Added Date,Num_Quiz_Attempt,Num_Correct,Num_Wrong,Last_Quiz
1,1,帮助,bang1 zhu4,bang1 zhu4,Noun/Verb,Support,Common,Help/Assistance,我可以帮助你学习中文,Wǒ kěyǐ bāngzhù nǐ xuéxí Zhōngwén.,I can help you study Chinese.,2024-09-21,0,0,0,
2,2,包裹,bao1 guo3,bao1 guo3,Noun,Object,Common,Package,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,2024-09-21,0,0,0,


In [3]:
quiz_generator = QuizGenerator(
    gsheet_name = "New Chinese Words",
    wks_name = "Tua_List"
    )

In [4]:
quiz_df = quiz_generator.generate_pinyin_and_meaning_quiz(
        num_words = 10, 
        date_filter = '2024-10-10', 
        #category_filter = 'Fashion',
        rarity_filter = 'Common'
        )

quiz_df_display = quiz_df.drop(columns=['Word Id'])
quiz_df_display

,Word,Sentence,Pinyin,Meaning
0,勉强,他勉强答应了我的请求。,,
1,追问,他不停地追问那个问题。,,
2,健康,健康是生活中最重要的财富之一。,,
3,乘客,车上有很多乘客。,,
4,服输,他在比赛中服输，表现得很大方。,,
5,托运,我们在机场托运了两件行李。,,
6,本,他买了一本书。,,
7,路灯,那条街的路灯全亮了。,,
8,演,他在学校的舞台上演了一场精彩的戏。,,
9,迁就,她总是迁就其他人的要求。,,


In [5]:
pinyin_answer = 'an1jing4, bie2, umm, ded, fan4, jiu2, yu4jian4, jiao3, you1mo4, sheng1bing4'.split(', ')
meaning_answer = 'quiet, dont, ded, to spin, to make a mistake, old, to meet, feet, humor, sick'.split(', ')

In [6]:
quiz_df_display['Pinyin'] = pinyin_answer
quiz_df_display['Meaning'] = meaning_answer

In [7]:
quiz_result = quiz_generator.evaluate_pinyin_and_meaning_quiz(
    pinyin_answer = quiz_df_display['Pinyin'], 
    meaning_answer = quiz_df_display['Meaning']
)

quiz_result

c:\Users\wongs\Documents\DS Work\repo\Pytorch LLM\GPT4LanguageLearning\main\quiz.py:125: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Word,Sentence,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction
Word Id,,,,,,,,
524,勉强,他勉强答应了我的请求。,an1jing4,no,miǎn qiǎng,quiet,no,"Incorrect meaning; it means ""to reluctantly ag..."
1469,追问,他不停地追问那个问题。,bie2,no,zhuī wèn,dont,no,"Incorrect meaning; it means ""to press for an a..."
1201,健康,健康是生活中最重要的财富之一。,umm,no,jiàn kāng,ded,no,"Incorrect meaning; it means ""health."""
796,乘客,车上有很多乘客。,ded,no,chéng kè,to spin,no,"Incorrect meaning; it means ""passenger."""
323,服输,他在比赛中服输，表现得很大方。,fan4,no,fú shū,to make a mistake,no,"Incorrect meaning; it means ""to admit defeat."""
650,托运,我们在机场托运了两件行李。,jiu2,no,tuō yùn,old,no,"Incorrect meaning; it means ""to check in lugga..."
423,本,他买了一本书。,yu4jian4,no,běn,to meet,no,"Incorrect meaning; it means ""copy/book."""
1411,路灯,那条街的路灯全亮了。,jiao3,no,lù dēng,feet,no,"Incorrect meaning; it means ""streetlight."""
1439,演,他在学校的舞台上演了一场精彩的戏。,you1mo4,no,yǎn,humor,no,"Incorrect meaning; it means ""to perform."""


In [8]:
quiz_generator.update_quiz_score(
    gsheet_name = gsheet_name, 
    wks_name = 'Tua_List'
    )

'Quiz Result Updated'

##### Check Update Condition

In [ ]:
quiz_result_df = quiz_result, 
gsheet_name = gsheet_name, 
wks_name = 'Tua_List'

word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=wks_name)

quiz_result_df['Right Score'] = np.where((quiz_result_df['Meaning Correct']=="yes")&(quiz_result_df['Pinyin Correct']=="yes"), 1, 0)
quiz_result_df['Wrong Score'] = np.where(quiz_result_df['Right Score']==0, 1, 0)
quiz_result_df['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

word_dict_quiz_export = word_dict.merge(quiz_result_df[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_df) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score', 'Last Quiz'])



## Manual Code

In [ ]:
quiz_result_export = quiz_result.copy()
quiz_result_export['Right Score'] = np.where((quiz_result_export['Meaning Correct']=="yes")&(quiz_result_export['Pinyin Correct']=="yes"), 1, 0)
quiz_result_export['Wrong Score'] = np.where(quiz_result_export['Right Score']==0, 1, 0)
quiz_result_export['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index()

In [ ]:
word_dict_quiz_export = word_dict.merge(quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_export) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score'])